CUDA_VISIBLE_DEVICES=0 python -m app.circor.solve_circor config/m2d.yaml circor1 weight_file=m2d_vit_base-80x608p16x16-221006-mr7/checkpoint-300.pth,encoder_only=True --lr=0.1 --freq_mask 0 --time_mask 0 --training_mask 0.0 --mixup 0.0 --rrc False --epochs 50 --warmup_epochs 0 --seed 8 --batch_size 1024 --hidden \(128,\) --reweight True

In [ ]:
import os
os.chdir('F:/github/evar')

config_file = 'config/m2d.yaml'
task = 'circor1'
options = 'weight_file=m2d_vit_base-80x608p16x16-221006-mr7/checkpoint-300.pth,encoder_only=True'
weight_file = 'm2d_vit_base-80x608p16x16-221006-mr7/checkpoint-300.pth'
unit_sec = None

from lineareval import make_cfg
cfg, n_folds, balanced = make_cfg(config_file, task, options, extras={}, abs_unit_sec=unit_sec)
print(cfg, n_folds, balanced)

from evar.data import create_dataloader
train_loader, _, _, _ = create_dataloader(cfg, fold=0, seed=0, batch_size=1, balanced_random=balanced, pin_memory=False)


d:\anaconda3\envs\evar\lib\site-packages\nnAudio\Spectrogram.py:4: Warning: importing Spectrogram subpackage will be deprecated soon. You should import the feature extractor from the feature subpackage. See actual documentation.
  warnings.warn(
d:\anaconda3\envs\evar\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\anaconda3\envs\evar\lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
d:\anaconda3\envs\evar\lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


weight_file=m2d_vit_base-80x608p16x16-221006-mr7/checkpoint-300.pth,encoder_only=True
+task_metadata=evar/metadata/circor1.csv,+task_data=work/16k/circor1,+unit_samples=80000

{'audio_repr': 'ar_m2d.AR_M2D', 'weight_file': 'm2d_vit_base-80x608p16x16-221006-mr7/checkpoint-300.pth', 'feature_d': 3840, 'sample_rate': 16000, 'n_fft': 400, 'window_size': 400, 'hop_size': 160, 'n_mels': 80, 'f_min': 50, 'f_max': 8000, 'window': 'hanning', 'mean': -7.1, 'std': 4.2, 'output_layers': [-1], 'encoder_only': True, 'dur_frames': None, 'freeze_embed': None, 'flat_features': False, 'batch_size': 128, 'lr_lineareval': 3e-05, 'report_per_epochs': 50, 'early_stop_epochs': 20, 'training_mask': 0.0, 'warmup_epochs': 5, 'mixup': 0.5, 'ft_bs': 128, 'ft_lr': 2.0, 'ft_early_stop_epochs': -1, 'ft_epochs': 200, 'ft_freq_mask': 30, 'ft_time_mask': 192, 'ft_noise': 0.0, 'ft_rrc': True, 'name': '', 'task_metadata': 'evar/metadata/circor1.csv', 'task_data': 'work/16k/circor1', 'unit_samples': 80000, 'id': 'm2d_vit_

In [8]:
import re
import pandas as pd

# 你上传的文件名
filename = './evar/metadata/circor3.csv'

import os
os.chdir('F:/github/evar')
# 读取文件
with open(filename, 'r', encoding='utf-8') as f:
    content = f.read()

# 用正则提取所有样本名（捕获患者编号和听诊位置）
pattern = re.compile(r'(\d+)_([MATP]V)_\d+\.wav', re.IGNORECASE)
patient_locs = {}

for m in pattern.finditer(content):
    patient = m.group(1)
    loc = m.group(2).upper()
    if patient not in patient_locs:
        patient_locs[patient] = set()
    patient_locs[patient].add(loc)

# 检查每个患者的听诊位置是否齐全
required_locs = {'MV', 'AV', 'TV', 'PV'}
have_all = {pid: locs for pid, locs in patient_locs.items() if required_locs == locs}
not_all = {pid: locs for pid, locs in patient_locs.items() if required_locs != locs}

print(f"总患者数: {len(patient_locs)}")
print(f"有全部4个听诊位置的患者数: {len(have_all)}")
print(f"缺失听诊位置的患者数: {len(not_all)}")
print("例子：缺失的患者及已有部位：")
for pid, locs in list(not_all.items())[:10]:
    print(f"患者{pid}: {locs}")

总患者数: 941
有全部4个听诊位置的患者数: 587
缺失听诊位置的患者数: 354
例子：缺失的患者及已有部位：
患者29378: {'AV', 'MV'}
患者38337: {'AV', 'MV'}
患者39456: {'PV', 'TV', 'MV'}
患者46532: {'AV'}
患者46778: {'MV'}
患者47002: {'AV', 'MV'}
患者49558: {'AV', 'MV'}
患者49572: {'PV', 'TV', 'MV'}
患者49595: {'AV', 'MV'}
患者49598: {'AV', 'MV'}


In [ ]:
import os
os.chdir('F:/github/evar')
from evar.data import WavDataset_paired
config_file = 'config/m2d.yaml'
task = 'circor1'
options = 'weight_file=m2d_vit_base-80x608p16x16-221006-mr7/checkpoint-300.pth,encoder_only=True'
weight_file = 'm2d_vit_base-80x608p16x16-221006-mr7/checkpoint-300.pth'
unit_sec = None

from lineareval import make_cfg
cfg, n_folds, balanced = make_cfg(config_file, task, options, extras={}, abs_unit_sec=unit_sec)

train_dataset = WavDataset_paired(cfg, 'train', holdout_fold=0, always_one_hot=True, random_crop=True)
print(train_dataset)


d:\anaconda3\envs\evar\lib\site-packages\nnAudio\Spectrogram.py:4: Warning: importing Spectrogram subpackage will be deprecated soon. You should import the feature extractor from the feature subpackage. See actual documentation.
  warnings.warn(
d:\anaconda3\envs\evar\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\anaconda3\envs\evar\lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
d:\anaconda3\envs\evar\lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


weight_file=m2d_vit_base-80x608p16x16-221006-mr7/checkpoint-300.pth,encoder_only=True
+task_metadata=evar/metadata/circor1.csv,+task_data=work/16k/circor1,+unit_samples=80000

Loaded 17746 records for train split


Found 611 patients in train split
Position segments count: {'MV': 5175, 'AV': 4367, 'PV': 4069, 'TV': 4111}
Missing position count: {'MV': 53, 'AV': 102, 'PV': 122, 'TV': 143}
self.multi_label:  False
Generated 6110 training samples (611 unique patients)
